# Stochastic Gradient Descent

In [1]:
%load_ext autoreload
%autoreload 2

# Global imports

In [2]:
import sys

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

# Local imports

In [3]:
sys.path.append('..')

from base_code.models.poc import NeuralNetwork as NN
from base_code.datasets import DryBeansDataset as DBD
from base_code.train_test import train, test

from base_code.preprocessing.nomalize_standarizer import pipeline as normalize_standarize
from base_code.dataloaders.base import ContinualLearningDataLoader as DataLoader

# Data preprocessing

In [4]:
dataset = DBD()

In [5]:
dataset.get_task_range(0, 2).labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [6]:
pipeline = normalize_standarize(dataset.features)

In [7]:
X = pipeline.transform(dataset.features)
dataset.features = X

# DataLoader settings

In [8]:
train_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Neural Netwrok

In [9]:
model = NN(features_shape=dataset.features_shape, output_shape=dataset.output_shape)

In [10]:
loss_fn = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train - Test

## Concurrent

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, train_dataloader, loss_fn)
print("Done!")

Epoch 1
-------------------------------
tensor([[ 5.6540e-02, -1.1788e-02,  2.0252e-01, -5.7454e-01,  1.0717e-01,
         -3.3371e-02, -5.0533e-02, -2.7293e-01, -7.5066e-01, -3.6967e-01,
         -5.6889e-01, -5.4597e-01, -3.0234e-01, -7.1445e-01, -6.6991e-01,
         -3.8234e-01],
        [ 2.7054e-01,  4.0839e-01, -1.6211e-01,  1.0891e+00, -2.4844e-01,
         -3.1871e-01, -2.7831e-01,  5.6460e-01,  7.0248e-01,  5.1522e-01,
          6.3810e-01,  8.4676e-01,  1.8342e-01,  1.1223e+00,  1.1244e+00,
          5.1986e-01],
        [ 2.7180e-01,  7.7477e-01,  6.9405e-01,  8.6585e-01,  6.7975e-01,
          8.2399e-01, -2.9159e-01,  8.6385e-01,  8.0996e-01,  8.6386e-01,
          9.1749e-01,  8.7278e-01,  7.9442e-01,  7.9750e-01,  8.4753e-01,
          8.6675e-01],
        [ 5.0860e-01,  1.0880e+00,  1.0848e+00,  1.0399e+00,  1.1480e+00,
          1.2810e+00, -5.3968e-01,  1.1537e+00,  1.0832e+00,  1.2203e+00,
          1.2435e+00,  1.1543e+00,  1.2533e+00,  1.0481e+00,  1.0628e+00,
   

TypeError: forward() takes 3 positional arguments but 4 were given

## Sequential

In [12]:
epochs = 5
for task_id in range(train_dataloader.get_tasks_length()):
    print(f"Task {task_id+1}\n-------------------------------")
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(model, train_dataloader.get_task(task_id), loss_fn, optimizer)
        test(model, train_dataloader.get_task_range(0, task_id+1), loss_fn)

Task 1
-------------------------------
Epoch 1
-------------------------------
loss: 1.276212  [   10/ 2027]
loss: 1.297114  [ 1010/ 2027]
loss: 1.187896  [ 2010/ 2027]
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.201651 

Epoch 2
-------------------------------
loss: 1.204704  [   10/ 2027]
loss: 1.175839  [ 1010/ 2027]
loss: 1.219845  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.4%, Avg loss: 1.185685 

Epoch 3
-------------------------------
loss: 1.166602  [   10/ 2027]
loss: 1.170341  [ 1010/ 2027]
loss: 1.174511  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.8%, Avg loss: 1.178281 

Epoch 4
-------------------------------
loss: 1.179792  [   10/ 2027]
loss: 1.168353  [ 1010/ 2027]
loss: 1.172774  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.9%, Avg loss: 1.174184 

Epoch 5
-------------------------------
loss: 1.165649  [   10/ 2027]
loss: 1.165880  [ 1010/ 2027]
loss: 1.173934  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.171874 

Task 2
-------------------------------
Epoch 1
---